In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
#from transformers import BertTokenizer, BertModel, AdamW, get_linear_schedule_with_warmup
from sklearn.model_selection import train_test_split

In [3]:
data = pd.read_csv('/content/drive/MyDrive/Spring2023/Deep Learning/Project/processed_pitch_data.csv')
data = pd.read_csv('/content/drive/MyDrive/Spring2023/Deep Learning/Project/processed_pitch_data.csv').drop('Unnamed: 0', axis=1)

In [4]:
data

,Stage,PDF URL,Industry,Processed Text
0,Series D,./BestPitchPDFs/bolt.pdf,Tech,bolt bolt growing scaled global network around...
1,Late Stage,./BestPitchPDFs/spotify.pdf,Entertainment,distributed computer science aes cry language ...
2,Late Stage,./BestPitchPDFs/wework.pdf,Real Estate,space service whore fundamental community cyst...
3,Pre-Seed,./BestPitchPDFs/airbnb.pdf,Travel and Hospitality,welcome breakfast book rather problem price im...
4,Early Stage,./BestPitchPDFs/facebook.pdf,Media and Advertising,ers sie aes oes see classes bee directory user...
...,...,...,...,...
1794,Seed,./InsiderPitchPDFs/Zestful.pdf,Enterprise,employee membership perk card real time track ...
1795,SeriesC,./InsiderPitchPDFs/Zeta.pdf,Finance and Banking,stack face banking zeta journey added bee bank...
1796,SeriesA,./InsiderPitchPDFs/Zevoy.pdf,Finance and Banking,efficient expense management regulated authori...
1797,Seed,./InsiderPitchPDFs/ZoeFinancial.pdf,Finance and Banking,matching mass affluent financial pee want fina...


In [5]:
data.columns = ['stage', 'url', 'industry', 'text']
data.head()

,stage,url,industry,text
0,Series D,./BestPitchPDFs/bolt.pdf,Tech,bolt bolt growing scaled global network around...
1,Late Stage,./BestPitchPDFs/spotify.pdf,Entertainment,distributed computer science aes cry language ...
2,Late Stage,./BestPitchPDFs/wework.pdf,Real Estate,space service whore fundamental community cyst...
3,Pre-Seed,./BestPitchPDFs/airbnb.pdf,Travel and Hospitality,welcome breakfast book rather problem price im...
4,Early Stage,./BestPitchPDFs/facebook.pdf,Media and Advertising,ers sie aes oes see classes bee directory user...


In [6]:
pitch_count = data.industry.value_counts()
pitch_count
low_count_industries = pitch_count[pitch_count < 50].index
# Remove industries with less than 50 pitches
data = data[~data.industry.isin(low_count_industries)]
data.industry.value_counts()

Finance and Banking              243
Health Care                      211
Enterprise                       207
Consumer                         196
Data Analytics and Management    172
Tech                             165
Media and Advertising            110
Entertainment                     83
Other                             70
Education                         67
Cybersecurity                     62
Real Estate                       53
Name: industry, dtype: int64

In [7]:
# Split data into train and test sets using cross validation
train, test = train_test_split(data, test_size=0.2, random_state=42, stratify=data.industry)
train.shape, test.shape

((1311, 4), (328, 4))

LTSM

In [10]:
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

MAX_NB_WORDS = 5000
MAX_SEQUENCE_LENGTH = 200
EMBEDDING_DIM = 100

# Convert all non-string values in the 'text' column to strings
train['text'] = train['text'].apply(lambda x: str(x) if not isinstance(x, str) else x)
test['text'] = test['text'].apply(lambda x: str(x) if not isinstance(x, str) else x)

# Tokenize and pad the text data
tokenizer = Tokenizer(num_words=MAX_NB_WORDS, filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~', lower=True)
tokenizer.fit_on_texts(train['text'].values)
word_index = tokenizer.word_index

X_train = tokenizer.texts_to_sequences(train['text'].values)
X_test = tokenizer.texts_to_sequences(test['text'].values)
X_train = pad_sequences(X_train, maxlen=MAX_SEQUENCE_LENGTH)
X_test = pad_sequences(X_test, maxlen=MAX_SEQUENCE_LENGTH)

# Encode the labels
encoder = LabelEncoder()
encoder.fit(train['industry'].values)
Y_train = to_categorical(encoder.transform(train['industry'].values))
Y_test = to_categorical(encoder.transform(test['industry'].values))


In [11]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Embedding, LSTM, SpatialDropout1D, Dense, Dropout

dropout_rate = 0.2

model = Sequential()
model.add(Embedding(MAX_NB_WORDS, EMBEDDING_DIM, input_length=X.shape[1]))
model.add(SpatialDropout1D(dropout_rate))
model.add(LSTM(256, dropout=dropout_rate, recurrent_dropout=dropout_rate))
model.add(Dense(Y_train.shape[1], activation='softmax'))
model.compile(loss="categorical_crossentropy", optimizer='adam', metrics=['accuracy'])

epochs = 20
batch_size = 64

history = model.fit(X_train, Y_train, epochs=epochs, batch_size=batch_size, validation_split=0.2)


Epoch 1/20
17/17 [==============================] - 51s 3s/step - loss: 2.4213 - accuracy: 0.1441 - val_loss: 2.3272 - val_accuracy: 0.1901
Epoch 2/20
17/17 [==============================] - 45s 3s/step - loss: 2.2940 - accuracy: 0.2481 - val_loss: 2.2242 - val_accuracy: 0.2167
Epoch 3/20
17/17 [==============================] - 45s 3s/step - loss: 2.0527 - accuracy: 0.3244 - val_loss: 2.0392 - val_accuracy: 0.3764
Epoch 4/20
17/17 [==============================] - 44s 3s/step - loss: 1.7690 - accuracy: 0.4294 - val_loss: 1.8472 - val_accuracy: 0.3650
Epoch 5/20
17/17 [==============================] - 46s 3s/step - loss: 1.5596 - accuracy: 0.5029 - val_loss: 1.7177 - val_accuracy: 0.3422
Epoch 6/20
17/17 [==============================] - 47s 3s/step - loss: 1.1937 - accuracy: 0.6422 - val_loss: 2.8594 - val_accuracy: 0.2471
Epoch 7/20
17/17 [==============================] - 44s 3s/step - loss: 1.2184 - accuracy: 0.6422 - val_loss: 1.6525 - val_accuracy: 0.4601
Epoch 8/20
17/17 [==

In [12]:
loss, accuracy = model.evaluate(X_test, Y_test, verbose=1)
print("Test accuracy: {:.2f}%".format(accuracy * 100))


11/11 [==============================] - 7s 600ms/step - loss: 2.0979 - accuracy: 0.5183
Test accuracy: 51.83%
